In [1]:

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import gc
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import pandas as pd
import os
from pandas.tseries.holiday import USFederalHolidayCalendar

from sklearn.model_selection import train_test_split
from pathlib import Path
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

from datetime import datetime

from winn_dixie_recpt_parser import WinnDixieRecptParser 

pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")

print(os.getcwd())


C:\Users\steve\OneDrive - NOLA Business IT\source\repos\grocery-ml


In [2]:
def show_grouped(grouped, rows=10):
    # collect only the daysSinceLastPurchase_* columns
    feature_cols = [c for c in grouped.columns if c.startswith("daysSinceLastPurchase_")]

    for i in range(min(rows, len(grouped))):
        print("Row:", i)
        print("Date:", grouped.iloc[i]["date"])
        print("Time:", grouped.iloc[i]["time"])
        print("Items:", grouped.iloc[i]["item"])
        print("------ daysSinceLastPurchase ------")

        for col in feature_cols:
            print(f"{col}: {grouped.iloc[i][col]}")

        print("-----------------------------------")


def show_encoded(encoded_df, rows=10):
    # Identify columns
    days_cols = [c for c in encoded_df.columns if c.startswith("daysSinceLastPurchase_")]
    weather_cols = [c for c in encoded_df.columns if c.endswith("_5day_avg")]
    item_cols = [
        c for c in encoded_df.columns 
        if c not in days_cols 
        and c not in weather_cols
        and c not in ["date", "time"]
    ]

    for i in range(min(rows, len(encoded_df))):
        print("Row:", i)
        print("Date:", encoded_df.iloc[i]["date"])
        print("Time:", encoded_df.iloc[i]["time"])

        # Show the items purchased (reverse one-hot)
        purchased_items = []
        row_vals = encoded_df.iloc[i]

        for item in item_cols:
            if row_vals[item] == 1:
                purchased_items.append(item)

        print("Items:", purchased_items)

        print("------ daysSinceLastPurchase ------")
        for col in days_cols:
            print(f"{col}: {encoded_df.iloc[i][col]}")

        print("------ weather (rolling windows) ------")
        for col in weather_cols:
            print(f"{col}: {encoded_df.iloc[i][col]}")

        print("-----------------------------------")


def remove_duplicate_receipt_files(df):
    """
    Remove whole source files that contain an identical receipt
    to another file with the same date+time.
    Minimal console output. Resets index at end.
    """

    df["__signature"] = (
        df["date"].astype(str) + "|" +
        df["time"].astype(str) + "|" +
        df["item"].astype(str) + "|" +
        df["qty"].astype(str) + "|" +
        df["youPay"].astype(str) + "|" +
        df["reg"].astype(str) + "|" +
        df["reportedItemsSold"].astype(str) + "|" +
        df["cashier"].astype(str) + "|" +
        df["manager"].astype(str)
    )

    keep_sources = set()

    for (dt_date, dt_time), group in df.groupby(["date", "time"]):

        # Build signature per source
        source_signatures = {}
        for source, rows in group.groupby("source"):
            sig = tuple(sorted(rows["__signature"].tolist()))
            source_signatures[source] = sig

        # signature → list of sources
        signature_groups = {}
        for src, sig in source_signatures.items():
            signature_groups.setdefault(sig, []).append(src)

        # Handle duplicates
        for sig, sources in signature_groups.items():
            if len(sources) == 1:
                keep_sources.add(sources[0])
                continue

            sorted_sources = sorted(sources)
            kept = sorted_sources[0]
            removed = sorted_sources[1:]

            # Minimal output
            print(f"DUP: {dt_date} {dt_time} → keep {kept} ← drop {', '.join(removed)}")

            keep_sources.add(kept)

    # Filter and clean
    result = df[df["source"].isin(keep_sources)].copy()
    result.drop(columns=["__signature"], inplace=True)

    # ✔ Reset index here
    result.reset_index(drop=True, inplace=True)

    return result



def rolling_freq(df, window_days):
    out = []
    for idx, row in df.iterrows():
        item = row["item"]
        cutoff = row["date"] - pd.Timedelta(days=window_days)
        count = df[(df["item"] == item) &
                   (df["date"] > cutoff) &
                   (df["date"] < row["date"])].shape[0]
        out.append(count)
    return out

In [3]:
def daysUntilNextHoliday(d):
    d = pd.to_datetime(d)
    holidays = USFederalHolidayCalendar().holidays()
    diffs = (holidays - d).days
    diffs = diffs[diffs >= 0]
    return diffs.min() if len(diffs) > 0 else np.nan
####################################################################

def daysSinceLastHoliday(d):
    d = pd.to_datetime(d)
    holidays = USFederalHolidayCalendar().holidays()
    diffs = (d - holidays).days
    diffs = diffs[diffs >= 0]
    return diffs.min() if len(diffs) > 0 else np.nan
####################################################################

def holidayProximityIndex(d, scale=30):
    """
    Returns a smooth value between -1 and +1 depending on
    distance to holidays. Neural networks LOVE this.
    Negative = after holiday
    Positive = before holiday
    """
    before = daysUntilNextHoliday(d)
    after = daysSinceLastHoliday(d)

    if pd.isna(before) and pd.isna(after):
        return 0

    # choose the nearest side (before or after)
    if before <= after:
        return +max(0, (scale - before) / scale)
    else:
        return -max(0, (scale - after) / scale)
####################################################################

def daysUntilBirthday(d, bday):
    d = pd.to_datetime(d)
    bday = pd.to_datetime(bday)

    this_year = pd.Timestamp(d.year, bday.month, bday.day)
    if d <= this_year:
        return (this_year - d).days
    else:
        next_year = pd.Timestamp(d.year + 1, bday.month, bday.day)
        return (next_year - d).days
####################################################################

def daysSinceBirthday(d, bday):
    d = pd.to_datetime(d)
    bday = pd.to_datetime(bday)

    this_year = pd.Timestamp(d.year, bday.month, bday.day)
    if d >= this_year:
        return (d - this_year).days
    else:
        last_year = pd.Timestamp(d.year - 1, bday.month, bday.day)
        return (d - last_year).days
####################################################################

def tempDeviation(actualTemp, avgTemp):
    """Signed deviation (continuous). Neural-network gold."""
    return actualTemp - avgTemp
####################################################################

def humidityDeviation(actualHumidity, avgHumidity):
    return actualHumidity - avgHumidity
####################################################################

def precipDeviation(actual, avg):
    return actual - avg
####################################################################

def daysUntilSchoolStart(d):
    d = pd.to_datetime(d)
    start = pd.Timestamp(d.year, 8, 15)
    if d <= start:
        return (start - d).days
    else:
        next_start = pd.Timestamp(d.year + 1, 8, 15)
        return (next_start - d).days
####################################################################

def daysUntilSchoolEnd(d):
    d = pd.to_datetime(d)
    end = pd.Timestamp(d.year, 5, 31)
    if d <= end:
        return (end - d).days
    else:
        next_end = pd.Timestamp(d.year + 1, 5, 31)
        return (next_end - d).days
####################################################################

def schoolSeasonIndex(d):
    """
    Smooth 0→1 curve inside school season.
    <0 before season, >1 after.
    Good for neural nets.
    """
    d = pd.to_datetime(d)
    start = pd.Timestamp(d.year, 8, 15)
    end   = pd.Timestamp(d.year, 5, 31)

    # If date is after Dec, school season continues in Jan–May.
    if d < start:
        return -((start - d).days) / 365.0
    elif start <= d <= end:
        return (d - start).days / (end - start).days
    else:
        return (d - end).days / 365.0

####################################################################


def normalizeAndDropCols(df, cols):
    for col in cols:
        # Replace the sentinel 999 with NaN so it doesn't distort mean/std
        df[col] = df[col].replace(999, np.nan)

        # Compute mean/std ignoring NaN
        mean = df[col].mean()
        std  = df[col].std() or 1.0

        # Normalize
        df[col + "_norm"] = (df[col] - mean) / std

        # After normalization: missing values become 0 (neutral)
        df[col + "_norm"] = df[col + "_norm"].fillna(0.0)

    return df.drop(columns=cols)


#def normalizeAndDropCols(df, cols):
#    for col in cols:
#        std = df[col].std() or 1.0
#        df[col + "_norm"] = (df[col] - df[col].mean()) / std
#    return df.drop(columns=cols)

In [4]:
# --- WEATHER PREP ---
weatherCols=["datetime", "temp", "humidity", "feelslike", "dew", "precip"]
df_weather = pd.read_csv("datasets/VisualCrossing-70062 2000-01-01 to 2025-12-14.csv", usecols=weatherCols)

df_weather["datetime"] = pd.to_datetime(df_weather["datetime"])
df_weather = df_weather.set_index("datetime").sort_index()

df_weather["temp_5day_avg"] = df_weather["temp"].rolling(5, min_periods=1).mean()
df_weather["feelsLike_5day_avg"] = df_weather["feelslike"].rolling(5, min_periods=1).mean()
df_weather["dew_5day_avg"] = df_weather["dew"].rolling(5, min_periods=1).mean()
df_weather["humidity_5day_avg"] = df_weather["humidity"].rolling(5, min_periods=1).mean()
df_weather["precip_5day_avg"] = df_weather["precip"].rolling(5, min_periods=1).mean()

df_weather = df_weather.drop(columns=["temp", "humidity", "feelslike", "dew", "precip"])

# convert index to date for merging
df_weather["date"] = df_weather.index.date
df_weather["date"] = pd.to_datetime(df_weather["date"])
df_weather = df_weather.set_index("date")

#grouped.to_csv("grouped.csv", index=False)
#grouped.info()


In [5]:
rows = []

recptParser  = WinnDixieRecptParser();

for p in Path("winndixie rcpts/StevePhone2/pdf/text").glob("*.txt"):
    result = recptParser.parse(p.read_text(encoding="utf-8", errors="ignore"))
    for r in result["items"]:
        rows.append({
            "source": p.name,
            "date": result["date"],
            "time": result["time"],
            "manager": result["manager"],
            "cashier": result["cashier"],
            "item": r["item"],
            "qty": r["qty"],
            "reg": r["reg"],
            "youPay": r["youPay"],
            "reportedItemsSold": result["reported"],
            #"rowsMatchReported": result["validation"]["rowsMatchReported"],
            "qtyMatchReported": result["validation"]["qtyMatchReported"],
        })

df_winndixie = pd.DataFrame(rows)

df_winndixie["date"] = pd.to_datetime(df_winndixie["date"])
df_winndixie["time"] = df_winndixie["time"].astype(str)
df_winndixie = remove_duplicate_receipt_files(df_winndixie)
df_winndixie = df_winndixie.sort_values(by=["date", "time"]).reset_index(drop=True)

DUP: 2025-02-19 00:00:00 7:20 PM → keep IMG_9734.txt ← drop IMG_9735.txt
DUP: 2025-04-08 00:00:00 11:50 AM → keep IMG_9723.txt ← drop IMG_9724.txt
DUP: 2025-05-15 00:00:00 8:19 PM → keep IMG_9713.txt ← drop IMG_9714.txt
DUP: 2025-08-02 00:00:00 10:29 PM → keep IMG_9693.txt ← drop IMG_9694.txt
DUP: 2025-10-07 00:00:00 6:06 PM → keep IMG_0017.txt ← drop IMG_9669.txt
DUP: 2025-10-14 00:00:00 6:08 PM → keep IMG_0014.txt ← drop IMG_9666.txt
DUP: 2025-10-17 00:00:00 9:23 PM → keep IMG_0012.txt ← drop IMG_9663.txt


In [6]:
### CREATE ITEM IDs
unique_items = sorted(df_winndixie["item"].unique())
item_to_id = {item: idx for idx, item in enumerate(unique_items)}
id_to_item = {idx: item for item, idx in item_to_id.items()}
df_winndixie["itemId"] = df_winndixie["item"].map(item_to_id)
df_winndixie.reset_index(drop=True, inplace=True)
df_winndixie.info()
df_winndixie.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   source             863 non-null    object        
 1   date               863 non-null    datetime64[ns]
 2   time               863 non-null    object        
 3   manager            644 non-null    object        
 4   cashier            775 non-null    object        
 5   item               863 non-null    object        
 6   qty                863 non-null    int64         
 7   reg                863 non-null    float64       
 8   youPay             863 non-null    float64       
 9   reportedItemsSold  863 non-null    int64         
 10  qtyMatchReported   863 non-null    bool          
 11  itemId             863 non-null    int64         
dtypes: bool(1), datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 75.1+ KB


,source,date,time,manager,cashier,item,qty,reg,youPay,reportedItemsSold,qtyMatchReported,itemId
0,IMG_9764.txt,2024-11-15,8:13 PM,None,STEPHEN,spaghettios-pasta,2,3.180000,2.500000,8,True,334
1,IMG_9764.txt,2024-11-15,8:13 PM,None,STEPHEN,mars-chocolate,1,6.490000,6.490000,8,True,189
2,IMG_9764.txt,2024-11-15,8:13 PM,None,STEPHEN,mandms-candies,1,1.790000,1.790000,8,True,188
3,IMG_9764.txt,2024-11-15,8:13 PM,None,STEPHEN,mandarins,1,6.990000,5.990000,8,True,187
4,IMG_9764.txt,2024-11-15,8:13 PM,None,STEPHEN,sparkling-ice-wtr,1,1.290000,1.000000,8,True,335
5,IMG_9764.txt,2024-11-15,8:13 PM,None,STEPHEN,sparkling-ice-wtr,1,1.290000,1.000000,8,True,335
6,IMG_9764.txt,2024-11-15,8:13 PM,None,STEPHEN,coca-cola,1,2.490000,2.490000,8,True,71
7,IMG_9763.txt,2024-11-22,9:15 PM,None,ACM LANE_092,hersheys-syrup,1,4.490000,4.490000,2,True,127
8,IMG_9763.txt,2024-11-22,9:15 PM,None,ACM LANE_092,mayfield-icecream,1,6.990000,3.500000,2,True,192
9,IMG_9762.txt,2024-11-24,3:40 PM,None,DIANA,bunny-bread,1,3.990000,3.990000,16,True,45


In [7]:

# Build full rctps
all_items = df_winndixie["itemId"].unique()
all_dates = df_winndixie["date"].unique()

full = (
    pd.MultiIndex.from_product([all_dates, all_items], names=["date", "itemId"])
    .to_frame(index=False)
)

# Merge
df_winndixie = full.merge(df_winndixie, on=["date", "itemId"], how="left")

# NOW create didBuy field safely
df_winndixie["didBuy"] = df_winndixie["qty"].notna().astype(int)


In [8]:
# 1. Build grouped table (one row per trip date)

grouped = ( df_winndixie[["date"]]
    .drop_duplicates()
    .sort_values("date")
    .reset_index(drop=True)
)

# 2. daysSinceLastTrip
grouped["daysSinceLastTrip"] = grouped["date"].diff().dt.days.fillna(0)


# 3. Holiday / Birthday / School features
grouped["daysUntilNextHoliday"] = grouped["date"].apply(daysUntilNextHoliday)
grouped["daysSinceLastHoliday"] = grouped["date"].apply(daysSinceLastHoliday)
grouped["holidayProximityIndex"] = grouped["date"].apply(holidayProximityIndex)
grouped["daysUntilSchoolStart"] = grouped["date"].apply(daysUntilSchoolStart)
grouped["daysUntilSchoolEnd"]   = grouped["date"].apply(daysUntilSchoolEnd)
grouped["schoolSeasonIndex"]    = grouped["date"].apply(schoolSeasonIndex)

dt = grouped["date"]
grouped["year"]    = dt.dt.year
grouped["month"]   = dt.dt.month
grouped["day"]     = dt.dt.day
grouped["dow"]     = dt.dt.dayofweek
grouped["doy"]     = dt.dt.dayofyear
grouped["quarter"] = dt.dt.quarter

steveBday  = "03-05-1980"
angieBday  = "08-11-1981"
maggieBday = "03-03-2016"
mil = "01-27-1962"

grouped["daysUntilBirthday_steve"] = grouped["date"].apply(lambda d: daysUntilBirthday(d, steveBday))
grouped["daysSinceBirthday_steve"] = grouped["date"].apply(lambda d: daysSinceBirthday(d, steveBday))


grouped["daysUntilBirthday_maggie"] = grouped["date"].apply(lambda d: daysUntilBirthday(d, maggieBday))
grouped["daysSinceBirthday_maggie"] = grouped["date"].apply(lambda d: daysSinceBirthday(d, maggieBday))

grouped["daysUntilBirthday_mil"] = grouped["date"].apply(lambda d: daysUntilBirthday(d, mil))
grouped["daysSinceBirthday_mil"] = grouped["date"].apply(lambda d: daysSinceBirthday(d, mil))

grouped["daysUntilBirthday_angie"] = grouped["date"].apply(lambda d: daysUntilBirthday(d, angieBday))
grouped["daysSinceBirthday_angie"] = grouped["date"].apply(lambda d: daysSinceBirthday(d, angieBday))


grouped["shopper"]    = grouped["date"].apply(schoolSeasonIndex)

# merge in weather
grouped = grouped.merge(df_weather, on="date", how="left")

df_winndixie = df_winndixie.merge(grouped, on="date", how="left")
df_winndixie.info()
df_winndixie.head(10)

df_winndixie.to_csv("df_merged_group_level_features.csv", index=False)

### DAYS SINCE LAST PURCHASED PER ITEM
# Must be sorted so diff() works correctly
df_winndixie = df_winndixie.sort_values(["itemId", "date"])

# Compute days since last purchase for each item
df_winndixie["daysSinceLastPurchase"] = (
    df_winndixie.groupby("itemId")["date"].diff().dt.days
)

# First-ever purchase of an item = no previous date
df_winndixie["daysSinceLastPurchase"] = df_winndixie["daysSinceLastPurchase"].fillna(999)

df_winndixie.to_csv("df_merged_daysSinceLastPurchased.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36917 entries, 0 to 36916
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      36917 non-null  datetime64[ns]
 1   itemId                    36917 non-null  int64         
 2   source                    863 non-null    object        
 3   time                      863 non-null    object        
 4   manager                   644 non-null    object        
 5   cashier                   775 non-null    object        
 6   item                      863 non-null    object        
 7   qty                       863 non-null    float64       
 8   reg                       863 non-null    float64       
 9   youPay                    863 non-null    float64       
 10  reportedItemsSold         863 non-null    float64       
 11  qtyMatchReported          863 non-null    object        
 12  didBuy            

In [9]:
# %%
### Frequency per item

#df_merged["freq_7"]  = rolling_freq(df_merged, 7)
#df_merged["freq_15"]  = rolling_freq(df_merged, 15)
#df_merged["freq_30"] = rolling_freq(df_merged, 30)
#df_merged["freq_90"] = rolling_freq(df_merged, 90)
#df_merged["freq_180"] = rolling_freq(df_merged, 180)
#df_merged["freq_365"] = rolling_freq(df_merged, 365)


#df_merged.to_csv("df_merged_with_freq.csv", index=False)
#df_merged.info()
#df_merged.head(500)

In [10]:
# Get all unique item names
#unique_items = grouped["item"].explode().unique()

# Map each item → index
#item_to_index = {item: idx for idx, item in enumerate(unique_items)}

#num_items = len(unique_items)
#vectors = []

# Build one-hot vector for each trip
#for item_list in grouped["item"]:
#    vector = np.zeros(num_items, dtype=np.int32)
#    for name in item_list:
#        vector[item_to_index[name]] = 1
#    vectors.append(vector)

# Convert to DataFrame (THIS was missing)
#encoded_items_df = pd.DataFrame(vectors, columns=unique_items)

In [11]:
### NORMALIZE TO ENCODED_DF
#freq_cols = [c for c in df_winndixie.columns if c.startswith("freq_")]
weather_cols = [c for c in df_winndixie.columns if c.endswith("_5day_avg")]
holiday_cols = [c for c in df_winndixie.columns if "holiday" in c.lower()]
school_cols = [c for c in df_winndixie.columns if "school" in c.lower()]
birthday_cols = [c for c in df_winndixie.columns if c.startswith("daysUntilBirthday_") or c.startswith("daysSinceBirthday_")]

daysSince_purchase_cols = ["daysSinceLastPurchase"]
daysSince_trip_cols     = ["daysSinceLastTrip"]


encoded_df = df_winndixie.copy()

#encoded_df = normalizeAndDropCols(encoded_df, freq_cols)
encoded_df = normalizeAndDropCols(encoded_df, weather_cols)
encoded_df = normalizeAndDropCols(encoded_df, holiday_cols)
encoded_df = normalizeAndDropCols(encoded_df, school_cols)
encoded_df = normalizeAndDropCols(encoded_df, birthday_cols)
encoded_df = normalizeAndDropCols(encoded_df, daysSince_purchase_cols)
encoded_df = normalizeAndDropCols(encoded_df, daysSince_trip_cols)

encoded_df.info();
encoded_df.head(100)


<class 'pandas.core.frame.DataFrame'>
Index: 36917 entries, 215 to 36553
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           36917 non-null  datetime64[ns]
 1   itemId                         36917 non-null  int64         
 2   source                         863 non-null    object        
 3   time                           863 non-null    object        
 4   manager                        644 non-null    object        
 5   cashier                        775 non-null    object        
 6   item                           863 non-null    object        
 7   qty                            863 non-null    float64       
 8   reg                            863 non-null    float64       
 9   youPay                         863 non-null    float64       
 10  reportedItemsSold              863 non-null    float64       
 11  qtyMatchReported  

,date,itemId,source,time,manager,cashier,item,qty,reg,youPay,reportedItemsSold,qtyMatchReported,didBuy,year,month,day,dow,doy,quarter,temp_5day_avg_norm,feelsLike_5day_avg_norm,dew_5day_avg_norm,humidity_5day_avg_norm,precip_5day_avg_norm,daysUntilNextHoliday_norm,daysSinceLastHoliday_norm,holidayProximityIndex_norm,daysUntilSchoolStart_norm,daysUntilSchoolEnd_norm,schoolSeasonIndex_norm,daysUntilBirthday_steve_norm,daysSinceBirthday_steve_norm,daysUntilBirthday_maggie_norm,daysSinceBirthday_maggie_norm,daysUntilBirthday_mil_norm,daysSinceBirthday_mil_norm,daysUntilBirthday_angie_norm,daysSinceBirthday_angie_norm,daysSinceLastPurchase_norm,daysSinceLastTrip_norm
215,2024-11-15,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,11,15,4,320,4,-0.002142,-0.143350,0.190003,0.708596,-0.201792,-0.467658,-0.872813,-1.241346,0.707201,-0.012630,1.169022,-0.596116,0.596116,-0.631442,0.631442,-0.924538,0.962680,0.644718,-0.613699,0.000000,-1.249063
595,2024-11-22,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,11,22,4,327,4,-0.475026,-0.514015,-0.765419,-1.205600,1.160189,-0.805867,-0.525219,1.134928,0.643589,-0.085362,1.219224,-0.668546,0.668546,-0.703468,0.703468,-0.992517,1.030213,0.580528,-0.549138,0.997862,1.006558
978,2024-11-24,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,11,24,6,329,4,-0.994532,-0.933930,-1.334843,-1.553636,-0.775622,-0.902498,-0.425907,1.229979,0.625414,-0.106142,1.233568,-0.689240,0.689240,-0.724047,0.724047,-1.011940,1.049508,0.562188,-0.530692,-0.617092,-0.604600
1358,2024-11-27,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,11,27,2,332,4,-0.623218,-0.642322,-0.269365,1.230649,-0.694634,-1.047445,-0.276938,1.372555,0.598152,-0.137313,1.255083,-0.720282,0.720282,-0.754915,0.754915,-1.041074,1.078451,0.534678,-0.503024,-0.294101,-0.282368
1738,2024-12-02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,12,2,0,337,4,-1.400813,-1.283858,-1.601213,-1.273694,-0.763770,0.015498,-0.872813,-1.241346,0.552714,-0.189264,1.290942,-0.772018,0.772018,-0.806362,0.806362,-1.089631,1.126690,0.488828,-0.456909,0.351880,0.362095
2119,2024-12-04,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,12,4,2,339,4,-1.605619,-1.451046,-1.712865,-0.971055,-0.764757,-0.081133,-0.773501,-1.146295,0.534539,-0.210045,1.305285,-0.792712,0.792712,-0.826940,0.826940,-1.109053,1.145985,0.470488,-0.438463,-0.617092,-0.604600
2499,2024-12-07,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,12,7,5,342,4,-1.587303,-1.479559,-1.625138,-0.736509,0.487594,-0.226080,-0.624532,-1.003719,0.507277,-0.241215,1.326800,-0.823754,0.823754,-0.857808,0.857808,-1.138187,1.174928,0.442978,-0.410794,-0.294101,-0.282368
2879,2024-12-12,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,12,12,3,347,4,-0.916273,-0.904121,-0.720758,0.524490,0.090555,-0.467658,-0.376251,0.802250,0.461839,-0.293166,1.362659,-0.875490,0.875490,-0.909255,0.909255,-1.186744,1.223166,0.397128,-0.364679,0.351880,0.362095
3259,2024-12-28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,12,28,5,363,4,-0.531639,-0.561968,0.060806,2.315108,2.208095,-0.902498,-0.922469,-1.288872,0.316439,-0.459410,1.477407,-1.041045,1.041045,-1.073885,1.073885,-1.342126,1.377529,0.250408,-0.217112,3.904780,3.906641
3639,2024-12-29,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2024,12,29,6,364,4,-0.521648,-0.554192,0.086326,2.388246,2.257478,-0.950814,-0.872813,1.277504,0.307352,-0.469800,1.484578,-1.051392,1.051392,-1.084175,1.084175,-1.351837,1.387176,0.241238,-0.207889,-0.940083,-0.926831


In [12]:
# ============================================================
# 4. SINE COSINE on CYCLICAL FEATURES
# ============================================================

encoded_df["dow_sin"] =   np.sin(2 * np.pi * encoded_df["dow"] / 7.0)
encoded_df["dow_cos"] =   np.cos(2 * np.pi * encoded_df["dow"] / 7.0)
encoded_df["month_sin"] = np.sin(2 * np.pi * encoded_df["month"] / 12.0)
encoded_df["month_cos"] = np.cos(2 * np.pi * encoded_df["month"] / 12.0)
encoded_df["doy_sin"] =   np.sin(2 * np.pi * encoded_df["doy"] / 365.0)
encoded_df["doy_cos"] =   np.cos(2 * np.pi * encoded_df["doy"] / 365.0)

encoded_df = encoded_df.drop(columns=["dow","month","doy"])

## NON-CYCLIC TIME FEATURES
nonCycCols = ["year","day","quarter"]
encoded_df = normalizeAndDropCols(encoded_df, nonCycCols)
#

cols_to_drop = ["source","manager","time","qty", "reg", "youPay", "reportedItemsSold", "qtyMatchReported"]
encoded_df = encoded_df.drop(columns=cols_to_drop, errors="ignore")

encoded_df.to_csv("encoded.csv", index=False)
encoded_df.info()
encoded_df.head(10)


<class 'pandas.core.frame.DataFrame'>
Index: 36917 entries, 215 to 36553
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           36917 non-null  datetime64[ns]
 1   itemId                         36917 non-null  int64         
 2   cashier                        775 non-null    object        
 3   item                           863 non-null    object        
 4   didBuy                         36917 non-null  int64         
 5   temp_5day_avg_norm             36917 non-null  float64       
 6   feelsLike_5day_avg_norm        36917 non-null  float64       
 7   dew_5day_avg_norm              36917 non-null  float64       
 8   humidity_5day_avg_norm         36917 non-null  float64       
 9   precip_5day_avg_norm           36917 non-null  float64       
 10  daysUntilNextHoliday_norm      36917 non-null  float64       
 11  daysSinceLastHolid

,date,itemId,cashier,item,didBuy,temp_5day_avg_norm,feelsLike_5day_avg_norm,dew_5day_avg_norm,humidity_5day_avg_norm,precip_5day_avg_norm,daysUntilNextHoliday_norm,daysSinceLastHoliday_norm,holidayProximityIndex_norm,daysUntilSchoolStart_norm,daysUntilSchoolEnd_norm,schoolSeasonIndex_norm,daysUntilBirthday_steve_norm,daysSinceBirthday_steve_norm,daysUntilBirthday_maggie_norm,daysSinceBirthday_maggie_norm,daysUntilBirthday_mil_norm,daysSinceBirthday_mil_norm,daysUntilBirthday_angie_norm,daysSinceBirthday_angie_norm,daysSinceLastPurchase_norm,daysSinceLastTrip_norm,dow_sin,dow_cos,month_sin,month_cos,doy_sin,doy_cos,year_norm,day_norm,quarter_norm
215,2024-11-15,0,NaN,NaN,0,-0.002142,-0.143350,0.190003,0.708596,-0.201792,-0.467658,-0.872813,-1.241346,0.707201,-0.012630,1.169022,-0.596116,0.596116,-0.631442,0.631442,-0.924538,0.962680,0.644718,-0.613699,0.000000,-1.249063,-0.433884,-0.900969,-0.500000,0.866025,-0.699458,0.714673,-2.796238,-0.130749,1.170430
595,2024-11-22,0,NaN,NaN,0,-0.475026,-0.514015,-0.765419,-1.205600,1.160189,-0.805867,-0.525219,1.134928,0.643589,-0.085362,1.219224,-0.668546,0.668546,-0.703468,0.703468,-0.992517,1.030213,0.580528,-0.549138,0.997862,1.006558,-0.433884,-0.900969,-0.500000,0.866025,-0.608477,0.793572,-2.796238,0.608808,1.170430
978,2024-11-24,0,NaN,NaN,0,-0.994532,-0.933930,-1.334843,-1.553636,-0.775622,-0.902498,-0.425907,1.229979,0.625414,-0.106142,1.233568,-0.689240,0.689240,-0.724047,0.724047,-1.011940,1.049508,0.562188,-0.530692,-0.617092,-0.604600,-0.781831,0.623490,-0.500000,0.866025,-0.580800,0.814046,-2.796238,0.820110,1.170430
1358,2024-11-27,0,NaN,NaN,0,-0.623218,-0.642322,-0.269365,1.230649,-0.694634,-1.047445,-0.276938,1.372555,0.598152,-0.137313,1.255083,-0.720282,0.720282,-0.754915,0.754915,-1.041074,1.078451,0.534678,-0.503024,-0.294101,-0.282368,0.974928,-0.222521,-0.500000,0.866025,-0.538005,0.842942,-2.796238,1.137063,1.170430
1738,2024-12-02,0,NaN,NaN,0,-1.400813,-1.283858,-1.601213,-1.273694,-0.763770,0.015498,-0.872813,-1.241346,0.552714,-0.189264,1.290942,-0.772018,0.772018,-0.806362,0.806362,-1.089631,1.126690,0.488828,-0.456909,0.351880,0.362095,0.000000,1.000000,-0.000000,1.000000,-0.463550,0.886071,-2.796238,-1.504213,1.170430
2119,2024-12-04,0,NaN,NaN,0,-1.605619,-1.451046,-1.712865,-0.971055,-0.764757,-0.081133,-0.773501,-1.146295,0.534539,-0.210045,1.305285,-0.792712,0.792712,-0.826940,0.826940,-1.109053,1.145985,0.470488,-0.438463,-0.617092,-0.604600,0.974928,-0.222521,-0.000000,1.000000,-0.432776,0.901502,-2.796238,-1.292911,1.170430
2499,2024-12-07,0,NaN,NaN,0,-1.587303,-1.479559,-1.625138,-0.736509,0.487594,-0.226080,-0.624532,-1.003719,0.507277,-0.241215,1.326800,-0.823754,0.823754,-0.857808,0.857808,-1.138187,1.174928,0.442978,-0.410794,-0.294101,-0.282368,-0.974928,-0.222521,-0.000000,1.000000,-0.385663,0.922640,-2.796238,-0.975958,1.170430
2879,2024-12-12,0,NaN,NaN,0,-0.916273,-0.904121,-0.720758,0.524490,0.090555,-0.467658,-0.376251,0.802250,0.461839,-0.293166,1.362659,-0.875490,0.875490,-0.909255,0.909255,-1.186744,1.223166,0.397128,-0.364679,0.351880,0.362095,0.433884,-0.900969,-0.000000,1.000000,-0.304921,0.952378,-2.796238,-0.447703,1.170430
3259,2024-12-28,0,NaN,NaN,0,-0.531639,-0.561968,0.060806,2.315108,2.208095,-0.902498,-0.922469,-1.288872,0.316439,-0.459410,1.477407,-1.041045,1.041045,-1.073885,1.073885,-1.342126,1.377529,0.250408,-0.217112,3.904780,3.906641,-0.974928,-0.222521,-0.000000,1.000000,-0.034422,0.999407,-2.796238,1.242714,1.170430
3639,2024-12-29,0,NaN,NaN,0,-0.521648,-0.554192,0.086326,2.388246,2.257478,-0.950814,-0.872813,1.277504,0.307352,-0.469800,1.484578,-1.051392,1.051392,-1.084175,1.084175,-1.351837,1.387176,0.241238,-0.207889,-0.940083,-0.926831,-0.781831,0.623490,-0.000000,1.000000,-0.017213,0.999852,-2.796238,1.348365,1.170430


In [13]:
# # ============================================================
# # 4. Trip-level features (one row per date)
# # ============================================================

# trip_cols = [
#     "temp_5day_avg_norm","feelsLike_5day_avg_norm","dew_5day_avg_norm",
#     "humidity_5day_avg_norm","precip_5day_avg_norm",
#     "daysUntilNextHoliday_norm","daysSinceLastHoliday_norm",
#     "holidayProximityIndex_norm","daysUntilSchoolStart_norm",
#     "daysUntilSchoolEnd_norm","schoolSeasonIndex_norm",
#     "daysUntilBirthday_steve_norm","daysSinceBirthday_steve_norm",
#     "daysUntilBirthday_angie_norm","daysSinceBirthday_angie_norm",
#     "daysUntilBirthday_maggie_norm","daysSinceBirthday_maggie_norm",
#     "daysUntilBirthday_mil_norm","daysSinceBirthday_mil_norm",
#     "dow_sin","dow_cos","month_sin","month_cos","doy_sin","doy_cos",
#     "year_norm","day_norm","quarter_norm",
#     "daysSinceLastTrip_norm"
# ]

# # Compact: trip-level DF = unique by date
# trip_df = encoded_df[["date"] + trip_cols].drop_duplicates("date")

# full_df = full_df.merge(trip_df, on="date", how="left")

# # ============================================================
# # 5. Item-level features (one row per item_id)
# # ============================================================

# item_cols = ["daysSinceLastPurchase_norm"]

# item_df = encoded_df[["item_id"] + item_cols].drop_duplicates("item_id")

# full_df = full_df.merge(item_df, on="item_id", how="left")

# # ============================================================
# # Done
# # ============================================================

# print("full_df shape:", full_df.shape)
# print(full_df.head(20))


# TRAIN / BUILD MODEL

In [14]:
# ============================================================
# FULLY SELF-CONTAINED TRAINING + SAVING METHODS (FINAL)
# ============================================================

import os, json
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split



# ============================================================
# TRAINING PROGRESS CALLBACK
# ============================================================
class EpochStatus(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        print(
            f"Epoch {epoch+1}: "
            f"loss={logs.get('loss'):.4f}  "
            f"val_loss={logs.get('val_loss'):.4f}  "
            f"acc={logs.get('accuracy'):.4f}"
        )

# ============================================================
# SAVE EXPERIMENT (MODEL, METRICS, HYPERPARAMS)
# ============================================================
def save_experiment(model, history, HP):

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    name = f"exp_e{HP['epochs']}_b{HP['batch_size']}_lr{HP['learning_rate']}"
    exp_dir = os.path.join("experiments", f"{timestamp}_{name}")
    os.makedirs(exp_dir, exist_ok=True)

    # Save hyperparameters
    with open(os.path.join(exp_dir, "hyperparams.json"), "w") as f:
        json.dump(HP, f, indent=4)

    # Save metrics CSV
    pd.DataFrame(history.history).to_csv(
        os.path.join(exp_dir, "training_metrics.csv"), index=False
    )

    # Save model architecture
    with open(os.path.join(exp_dir, "model_arch.json"), "w") as f:
        f.write(model.to_json())

    # Save weights
    model.save_weights(os.path.join(exp_dir, "model.weights.h5"))

    print(f"\nSaved experiment to: {exp_dir}")

    return exp_dir


In [15]:
# ============================================================
# EXPERIMENT LOOP — RUN MULTIPLE TRAINING CONFIGS
# ============================================================

BATCH_SIZE     = 32
LEARNING_RATE  = 0.001

EPOCHS_START   = 5
EPOCHS_STEP    = 10
EPOCHS_MAX     = 500

epochs = EPOCHS_START

while epochs <= EPOCHS_MAX:

    print(f"\n==============================")
    print(f" RUNNING EXPERIMENT: epochs={epochs}")
    print(f"==============================")

    HP = {
        "epochs": epochs,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE
    }

    epochs += EPOCHS_STEP

    # ---- Features from encoded_df ----
    feature_cols = [
        c for c in encoded_df.columns
        if c != "didBuy" and np.issubdtype(encoded_df[c].dtype, np.number)
    ]

    # ---- 1. X and y ----
    X = encoded_df[feature_cols].to_numpy(dtype=np.float32)
    y = encoded_df["didBuy"].to_numpy(dtype=np.float32)

    # ---- 2. Split ----
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=True, random_state=42
    )

    # ---- 3. Model ----
    model = models.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(128, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=HP["learning_rate"]),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    # ---- 4. Class weights ----
    pos = y_train.sum()
    neg = len(y_train) - pos
    cw = min(50.0, neg / (pos + 1e-6))
    class_weights = {0: cw}

    # ---- 5. Train ----
    history = model.fit(
        X_train, y_train,
        epochs=HP["epochs"],
        batch_size=HP["batch_size"],
        validation_split=0.1,
        class_weight=class_weights,
        verbose=0,
        callbacks=[EpochStatus()]
    )

    # ---- 6. Save results ----
    save_experiment(model=model, history=history, HP=HP)



 RUNNING EXPERIMENT: epochs=5
Epoch 1: loss=0.7711  val_loss=0.1694  acc=0.9745
Epoch 2: loss=0.2169  val_loss=0.1809  acc=0.9757
Epoch 3: loss=0.2150  val_loss=0.1981  acc=0.9757
Epoch 4: loss=0.2161  val_loss=0.1642  acc=0.9757
Epoch 5: loss=0.2164  val_loss=0.1750  acc=0.9757

Saved experiment to: experiments\20251208_210308_exp_e5_b32_lr0.001

 RUNNING EXPERIMENT: epochs=15
Epoch 1: loss=0.3930  val_loss=0.2526  acc=0.9757
Epoch 2: loss=0.2414  val_loss=0.1482  acc=0.9757
Epoch 3: loss=0.2296  val_loss=0.2112  acc=0.9757
Epoch 4: loss=0.2233  val_loss=0.1668  acc=0.9757
Epoch 5: loss=0.2209  val_loss=0.1685  acc=0.9757
Epoch 6: loss=0.2122  val_loss=0.1477  acc=0.9757
Epoch 7: loss=0.2127  val_loss=0.1522  acc=0.9757
Epoch 8: loss=0.2105  val_loss=0.1897  acc=0.9757
Epoch 9: loss=0.2092  val_loss=0.1777  acc=0.9757
Epoch 10: loss=0.2079  val_loss=0.1885  acc=0.9757
Epoch 11: loss=0.2067  val_loss=0.1736  acc=0.9757
Epoch 12: loss=0.2074  val_loss=0.1885  acc=0.9757
Epoch 13: loss=

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selec

Epoch 309: loss=0.1848  val_loss=0.2095  acc=0.9770
Epoch 310: loss=0.1869  val_loss=0.2056  acc=0.9769
Epoch 311: loss=0.1863  val_loss=0.2143  acc=0.9772
Epoch 312: loss=0.1898  val_loss=0.1976  acc=0.9763
Epoch 313: loss=0.1893  val_loss=0.1995  acc=0.9763
Epoch 314: loss=0.1889  val_loss=0.2021  acc=0.9763
Epoch 315: loss=0.1890  val_loss=0.2029  acc=0.9764
Epoch 316: loss=0.1880  val_loss=0.2114  acc=0.9764
Epoch 317: loss=0.1867  val_loss=0.2054  acc=0.9767
Epoch 318: loss=0.1851  val_loss=0.2053  acc=0.9769
Epoch 319: loss=0.1837  val_loss=0.2076  acc=0.9772
Epoch 320: loss=0.1820  val_loss=0.2089  acc=0.9775
Epoch 321: loss=0.1865  val_loss=0.2052  acc=0.9772
Epoch 322: loss=0.1912  val_loss=0.2173  acc=0.9765
Epoch 323: loss=0.1908  val_loss=0.2386  acc=0.9764
Epoch 324: loss=0.1876  val_loss=0.2156  acc=0.9767
Epoch 325: loss=0.1840  val_loss=0.2112  acc=0.9770
Epoch 326: loss=0.1824  val_loss=0.2102  acc=0.9772
Epoch 327: loss=0.1825  val_loss=0.2123  acc=0.9772
Epoch 328: l

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selec

Epoch 343: loss=0.1855  val_loss=0.2310  acc=0.9768
Epoch 344: loss=0.1857  val_loss=0.2294  acc=0.9768
Epoch 345: loss=0.1835  val_loss=0.2176  acc=0.9772
Epoch 346: loss=0.1853  val_loss=0.2271  acc=0.9770
Epoch 347: loss=0.1861  val_loss=0.2211  acc=0.9768
Epoch 348: loss=0.1818  val_loss=0.2249  acc=0.9774
Epoch 349: loss=0.1842  val_loss=0.2302  acc=0.9774
Epoch 350: loss=0.1880  val_loss=0.2240  acc=0.9766
Epoch 351: loss=0.1840  val_loss=0.2154  acc=0.9769
Epoch 352: loss=0.1836  val_loss=0.2134  acc=0.9770
Epoch 353: loss=0.1847  val_loss=0.2163  acc=0.9770
Epoch 354: loss=0.1832  val_loss=0.2205  acc=0.9772
Epoch 355: loss=0.1855  val_loss=0.2250  acc=0.9767
Epoch 356: loss=0.1835  val_loss=0.2133  acc=0.9772
Epoch 357: loss=0.1828  val_loss=0.2196  acc=0.9772
Epoch 358: loss=0.1915  val_loss=0.2288  acc=0.9771
Epoch 359: loss=0.1877  val_loss=0.2232  acc=0.9769
Epoch 360: loss=0.1839  val_loss=0.2194  acc=0.9772
Epoch 361: loss=0.1837  val_loss=0.2188  acc=0.9771
Epoch 362: l

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selec

Epoch 433: loss=0.1848  val_loss=0.2839  acc=0.9767
Epoch 434: loss=0.1835  val_loss=0.3081  acc=0.9768
Epoch 435: loss=0.1865  val_loss=0.2961  acc=0.9764
Epoch 436: loss=0.1851  val_loss=0.2848  acc=0.9766
Epoch 437: loss=0.1828  val_loss=0.2927  acc=0.9767
Epoch 438: loss=0.1808  val_loss=0.3000  acc=0.9769
Epoch 439: loss=0.1829  val_loss=0.2776  acc=0.9769
Epoch 440: loss=0.1814  val_loss=0.2984  acc=0.9769
Epoch 441: loss=0.1810  val_loss=0.2936  acc=0.9770
Epoch 442: loss=0.1850  val_loss=0.2991  acc=0.9768
Epoch 443: loss=0.2026  val_loss=0.3174  acc=0.9766
Epoch 444: loss=0.1866  val_loss=0.3215  acc=0.9765
Epoch 445: loss=0.1839  val_loss=0.3102  acc=0.9766
Epoch 446: loss=0.1830  val_loss=0.3099  acc=0.9766
Epoch 447: loss=0.1853  val_loss=0.3225  acc=0.9765
Epoch 448: loss=0.1855  val_loss=0.3139  acc=0.9764
Epoch 449: loss=0.1830  val_loss=0.3149  acc=0.9767
Epoch 450: loss=0.1819  val_loss=0.3024  acc=0.9768
Epoch 451: loss=0.1819  val_loss=0.3035  acc=0.9769
Epoch 452: l

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selec

Epoch 468: loss=0.1856  val_loss=0.3095  acc=0.9764
Epoch 469: loss=0.1847  val_loss=0.3004  acc=0.9766
Epoch 470: loss=0.1827  val_loss=0.3023  acc=0.9767
Epoch 471: loss=0.1826  val_loss=0.3125  acc=0.9768
Epoch 472: loss=0.1835  val_loss=0.3040  acc=0.9767
Epoch 473: loss=0.1841  val_loss=0.3008  acc=0.9767
Epoch 474: loss=0.1874  val_loss=0.3052  acc=0.9767
Epoch 475: loss=0.1851  val_loss=0.3133  acc=0.9764
Epoch 476: loss=0.1828  val_loss=0.3219  acc=0.9770
Epoch 477: loss=0.1878  val_loss=0.3076  acc=0.9763
Epoch 478: loss=0.1858  val_loss=0.3198  acc=0.9762
Epoch 479: loss=0.1857  val_loss=0.3136  acc=0.9764
Epoch 480: loss=0.1852  val_loss=0.3040  acc=0.9764
Epoch 481: loss=0.1855  val_loss=0.3014  acc=0.9764
Epoch 482: loss=0.1850  val_loss=0.2982  acc=0.9766
Epoch 483: loss=0.1816  val_loss=0.3178  acc=0.9770
Epoch 484: loss=0.1867  val_loss=0.3126  acc=0.9768
Epoch 485: loss=0.1869  val_loss=0.2943  acc=0.9763
Epoch 486: loss=0.1854  val_loss=0.2995  acc=0.9764
Epoch 487: l

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selector_events.py", line 132, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\events.py", line 89, in _run
    self._context.run(self._callback, *self._args)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\steve\.conda\envs\intercard-ml\Lib\asyncio\selec

In [ ]:
# Get predictions for the test set
y_pred = model.predict(X_test)

# Apply threshold
threshold = 0.5
y_pred_bin = (y_pred >= threshold).astype(int)


#pd.DataFrame(y_pred_bin, columns=frequent_items).head(100)

In [ ]:

def predict_next_trip(model, encoded_df, input_feature_cols, frequent_items):
    """
    Build a single input row for prediction using the most recent trip in encoded_df.
    Returns a sorted DataFrame of predicted probabilities.
    """

    # ------------------------------
    # 1. Get the most recent row (latest trip)
    # ------------------------------
    last = encoded_df.iloc[-1]

    # ------------------------------
    # 2. Build a new row using last-known feature values
    # ------------------------------
    x = {}

    for col in input_feature_cols:
        if col in encoded_df.columns:
            x[col] = last[col]
        else:
            # safety: unknown column
            x[col] = 0.0

    # Convert to model input shape
    X_input = np.array([x[col] for col in input_feature_cols], dtype=np.float32)
    X_input = X_input.reshape(1, -1)

    # ------------------------------
    # 3. Predict probabilities
    # ------------------------------
    y_pred = model.predict(X_input)[0]   # shape: (num_items,)

    # ------------------------------
    # 4. Build labeled output table
    # ------------------------------
    result = pd.DataFrame({
        "item": frequent_items,
        "probability": y_pred
    })

    # Sort highest-probability first
    result = result.sort_values(by="probability", ascending=False)

    return result


In [ ]:
pred = predict_next_trip(
    model=model,
    encoded_df=encoded_df,
    input_feature_cols=input_feature_cols,
    frequent_items=frequent_items
)

print(pred.head(20))